---  
title: "軌道最適化に関するノートブック"  
author: "Naoya Ozaki"  
date: "24 January 2024"  
output: "trajectory_optimization"  

---  

# Julia言語による軌道最適化に関するノートブック

## 1. SPICEの使い方

本章では，NASAジェット推進研究所(JPL)のNavigation and Ancillary Information Facility (NAIF)が提供するSPICE 1)の使い方をまとめる．C,FOTRAN, MATLAB等のインタフェースが提供されているが，ここではJuliaインタフェースである`SPICE.jl` 2)を用いて，惑星の精密な軌道情報を取得する．

1. https://naif.jpl.nasa.gov/naif/
2. https://github.com/JuliaAstro/SPICE.jl

In [1]:
import SPICE

# Load generic kernels
SPICE.furnsh("/Users/naoya/Programming/SPICE/generic_kernels/lsk/naif0012.tls") # Leap seconds kernel
SPICE.furnsh("/Users/naoya/Programming/SPICE/generic_kernels/pck/gm_de431.tpc") # Gravity Constant
SPICE.furnsh("/Users/naoya/Programming/SPICE/generic_kernels/spk/planets/de440.bsp") # Planetary ephemeris kernel

# Convert the calendar date to ephemeris seconds past J2000
et = SPICE.utc2et("2024-01-24T17:00:00")

# Get the position of Mars at `et` w.r.t. Earth
SPICE.spkezr("MOON", et, "J2000", "NONE", "EARTH")

([-149729.32542123165, 323380.9514241508, 178003.47653310827, -0.930096599343289, -0.31477987020879544, -0.13948942418980217], 1.3287384459430396)

## 2. `DifferentialEquations.jl`による軌道伝播

In [2]:
using DifferentialEquations
using LinearAlgebra

"""
   eom_nbp!(dxdt, x, p, t)

Computes the equation of motion for the N-body problem.

# Arguments
- `dxdt`: The derivative of the state vector.
- `x`: The state vector.
- `p`: The parameter sets.
- `t`: The time.
"""
function eom_nbp!(dxdt, x, p, t)
    # Initialize dx/dt
    dxdt[1:3] .= x[4:6]
    dxdt[4:6] .= p.u

    # Calculate Gravitational Acceleration
    for id in p.list_bodies
        # Get planetary ephemeris
        x_body = SPICE.spkez(id, t, "J2000", "NONE", 0)[1]
        r_body = x_body[1:3]
        #v_body = x_body[4:6] / (p.lsf / p.tsf)
        gm_body = SPICE.bodvcd(id, "GM", 1)[1]

        # Acceleration due to the body id
        r_rel = x[1:3] - r_body
        dxdt[4:6] .-= gm_body * r_rel / norm(r_rel)^3
    end
end

eom_nbp!

In [4]:
# Propagator Setting
et0 = SPICE.utc2et("2024-01-24T17:00:00")
x0 = SPICE.spkez(301, et0, "J2000", "NONE", 0)[1]
tspan = (et0, et0+1000.0)
prob = ODEProblem(eom_nbp!, x0, tspan, (;list_bodies=[10, 399, 5], u=[1.0e-6, 0.0, 0.0]))

# Solve
sol = solve(prob, Vern7(), reltol=1e-15, abstol=1e-15)

retcode: Success
Interpolation: specialized 7th order lazy interpolation
t: 31-element Vector{Float64}:
 7.593876691845803e8
 7.593876692166234e8
 7.593876692872773e8
 7.593876694114087e8
 7.59387669637369e8
 7.593876700521226e8
 7.593876706983421e8
 7.593876717407582e8
 7.59387673422188e8
 7.593876761428325e8
 7.593876797745717e8
 7.593876841822865e8
 7.593876900026442e8
 ⋮
 7.593878009000132e8
 7.593878383801446e8
 7.59387878904786e8
 7.593879270270623e8
 7.593879800603442e8
 7.593880604357818e8
 7.593881367942666e8
 7.59388227419768e8
 7.593883390807638e8
 7.593884529895371e8
 7.59388573837933e8
 7.593886691845803e8
u: 31-element Vector{Vector{Float64}}:
 [-8.337281530069983e7, 1.1208039905539091e8, 4.86573716433273e7, -26.13075980960512, -15.653048739955954, -6.788327413072386]
 [-8.33728161380097e7, 1.120803985538191e8, 4.865737142580842e7, -26.130759638691767, -15.653048953961287, -6.788327513533659]
 [-8.337281798425062e7, 1.1208039744786961e8, 4.865737094618638e7, -26.130759261

## 3. JuMP, Ipopt.jlを使った最適化

In [5]:
using JuMP, Ipopt
model = Model(Ipopt.Optimizer)
set_attribute(model, "max_cpu_time", 60.0)
set_attribute(model, "print_level", 0)